In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.profiler import profile, ProfilerActivity
import os
from google.colab import drive
drive.mount('/content/drive')  # te pedirá permisos [web:50]
import torch_xla.core.xla_model as xm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
device = xm.xla_device()   # devuelve el dispositivo XLA (TPU)
print("Device:", device)

Device: xla:0


/tmp/ipython-input-1215285935.py:1: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()   # devuelve el dispositivo XLA (TPU)


In [8]:
N_SAMPLES = 500
N_FEATURES = 128
N_CLASSES = 10

X = torch.randn(N_SAMPLES, N_FEATURES)          # [N, 128]
y = torch.randint(0, N_CLASSES, (N_SAMPLES,))   # [N]

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

model = nn.Sequential(
    nn.Linear(128,256 ),
    nn.ReLU(),         # [N, 64*8*8] → [N, 64, 8, 8]
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,10)
)
model = model .to(device)

loss_fn= nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


activities = [ProfilerActivity.CPU]
with profile(activities=activities, with_stack=True, with_modules=True) as prof:
    for features, targets in dataloader:
      features = features.to(device)
      targets = targets.to(device)
      optimizer.zero_grad()
      outputs = model(features)
      loss = loss_fn(outputs, targets)
      loss.backward()
      optimizer.step()

# Ruta directa al Escritorio (home/Desktop) y creación de carpeta si existe
drive_dir = "/content/drive/MyDrive/Colab Notebooks"
os.makedirs(drive_dir, exist_ok=True)

trace_path = os.path.join(drive_dir, "trace.json")
prof.export_chrome_trace(trace_path)  # escribe el archivo en esa ruta [web:47]